In [3]:
from Crypto.Util.number import long_to_bytes, bytes_to_long, GCD, getPrime, inverse
import random

# Prerequisites

- Diffie hellman
- Discrete logarithm Problem 
- Cyclic groups

# Theory

https://www.youtube.com/watch?v=6ARDqCckjfs

- Elgamal encryption is an public-key cryptosystem. It uses asymmetric key encryption for communicating between two parties and encrypting the message.
- This cryptosystem is based on the difficulty of finding discrete logarithm in a cyclic group that is even if we know $g^a$ and $g^b$, it is extremely difficult to compute $g^{ab}$.
- Elgamal is based on DLP. The same vulnerabilities apply here too

Parameter creation
- Choose large prime $p$. Choose $g$ with large prime order $q$ (generator of $\mathbb{Z}_p^*$)

Key generation(Alice):  
- Choose private key $1 \leq a \leq p-1$ Compute $A=g^{a}(\bmod p)$  
- Publish the public key $(g, A, p)$

Encryption(Bob):  
-  Choose plaintext $m \in \{0, 1, \dots, p-1\}$ 
- Choose random element $1 \leq k \leq p-2$ 
- Use Alice's public key $A$ to compute 
    - $c_{1} \equiv g^{k} \bmod p $  = ephemeral key (random each time)
    - $c_{2} \equiv m A^{k} \equiv m \cdot g^{ak} \bmod p$
  
- Send ciphertext $\left(c_{1}, c_{2}\right)$ to Alice.

Decryption(Alice):  
- Compute $\left(c_{1}^{a}\right)^{-1} \cdot c_{2}(\bmod p)$
- This quantity is equal to $m$

**Decryption Proof**:

$x \equiv\left(c_{1}^{a}\right)^{-1}(\bmod p)$

$\begin{aligned} x \cdot c_{2} & \equiv\left(c_{1}^{a}\right)^{-1} \cdot c_{2} &(\bmod p), & \text { since } x \equiv\left(c_{1}^{a}\right)^{-1}(\bmod p) \\ & \equiv\left(g^{a k}\right)^{-1} \cdot\left(m A^{k}\right) &(\bmod p), & \text { since } c_{1} \equiv g^{k}, c_{2} \equiv m A^{k}(\bmod p) \\ & \equiv\left(g^{a k}\right)^{-1} \cdot\left(m\left(g^{a}\right)^{k}\right)(\bmod p), & & \text { since } A \equiv g^{a}(\bmod p) \end{aligned}$
$\equiv m$
$(\bmod p)$
since the $g^{a k}$ terms cancel out.

**Remark**:
- $+$ since $k$ is random every time => encrypting the same message twice will output the different results
- $-$ a general ElGamal encryption produces a 2:1 expansion in size from plaintext to ciphertext (we need c1 and c2).
- $\pm$ Encryption under ElGamal requires two exponentiations; however, these exponentiations are independent of the message (precomputed) and can be computed ahead of time if need be

**Security**
- DLP weaknesses (If we have an DLP algorithm)
    - we can compute $a$ from $g^a \equiv A \bmod p$ => we can compute the shared key $g^{ak}$ (ephemeral key) => we can decrypt
    - we can compute $k$

# Code

In [9]:
def gen_params():
    p = getPrime(1024)
    g = 2
    return g, p


def gen_keys(g, p):
    a = random.randint(1, p-1)
    return pow(g, a, p), a


def elg_encryption(m, A, g, p):
    k = random.randint(1, p-1)
    c1 = pow(g, k, p)
    c2 = (m * pow(A, k, p))% p
    return c1, c2

def elg_decryption(c1, c2, a, p):
    x = inverse(pow(c1, a, p), p)
    m_decr =(x * c2) % p
    return m_decr

In [10]:
g, p = gen_params()
A, a = gen_keys(g, p)

In [11]:
m = bytes_to_long(b'secret_message')
c1, c2 = elg_encryption(m, A, g, p)
m_decr = elg_decryption(c1, c2, a, p)

In [14]:
print(m_decr == m)
print(long_to_bytes(m_decr))

True
b'secret_message'


# Resources

- https://github.com/ashutosh1206/Crypton/tree/master/Elgamal-Encryption
- https://en.wikipedia.org/wiki/ElGamal_encryption
- https://mathstats.uncg.edu/sites/pauli/112/HTML/secelgamal.html
- http://cryptowiki.net/index.php?title=El_Gamal_Scheme
- https://www.youtube.com/watch?v=pyirxbHuvOw